In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import multiprocessing

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

root_dir = os.getcwd().replace('notebooks', '')
# sys.path.append(root_dir)
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"   

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

In [3]:
n_cpus = multiprocessing.cpu_count()
n_cpus

24

In [4]:
n_gpus = len(tf.config.list_physical_devices('GPU'))
n_gpus

3

In [5]:
from env_crypto import CryptoEnv

C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [6]:
pairs = 'ADAUSDT'
lag = 150

In [7]:
url = f'C:/Users/yosou/Documents/BooksWithCode/Binance/Binance-Ai-Bot-Starter/Data/SPOT/{pairs}/{pairs}-5m-data.csv'

In [8]:
data = pd.read_csv(url)
data

,timestamp,open,high,low,close,volume
0,2021-01-01 00:00:00,0.1813,0.1815,0.1804,0.1805,1719341.2000
1,2021-01-01 00:05:00,0.1805,0.1806,0.1791,0.1793,3551263.8000
2,2021-01-01 00:10:00,0.1793,0.1796,0.1783,0.1786,3737301.1000
3,2021-01-01 00:15:00,0.1787,0.1798,0.1785,0.1797,1539790.0000
4,2021-01-01 00:20:00,0.1797,0.1797,0.1789,0.1794,945063.5000
...,...,...,...,...,...,...
68198,2021-08-26 09:35:00,2.5950,2.5970,2.5850,2.5870,610883.2000
68199,2021-08-26 09:40:00,2.5860,2.5880,2.5820,2.5870,571969.7000
68200,2021-08-26 09:45:00,2.5860,2.5870,2.5690,2.5780,1485829.2000
68201,2021-08-26 09:50:00,2.5780,2.5820,2.5540,2.5580,2166139.3000


In [9]:
data = data.set_index('timestamp')
data.index = pd.to_datetime(data.index)
data =  data.astype(float)
data

,open,high,low,close,volume
timestamp,,,,,
2021-01-01 00:00:00,0.1813,0.1815,0.1804,0.1805,1719341.2000
2021-01-01 00:05:00,0.1805,0.1806,0.1791,0.1793,3551263.8000
2021-01-01 00:10:00,0.1793,0.1796,0.1783,0.1786,3737301.1000
2021-01-01 00:15:00,0.1787,0.1798,0.1785,0.1797,1539790.0000
2021-01-01 00:20:00,0.1797,0.1797,0.1789,0.1794,945063.5000
...,...,...,...,...,...
2021-08-26 09:35:00,2.5950,2.5970,2.5850,2.5870,610883.2000
2021-08-26 09:40:00,2.5860,2.5880,2.5820,2.5870,571969.7000
2021-08-26 09:45:00,2.5860,2.5870,2.5690,2.5780,1485829.2000


In [10]:
data = data.iloc[64500:]
data

,open,high,low,close,volume
timestamp,,,,,
2021-08-13 13:25:00,2.0372,2.0402,2.0311,2.0312,1517479.9100
2021-08-13 13:30:00,2.0312,2.0369,2.0261,2.0297,1857518.4300
2021-08-13 13:35:00,2.0297,2.0312,2.0221,2.0238,2746010.5000
2021-08-13 13:40:00,2.0240,2.0255,2.0178,2.0198,1765897.1200
2021-08-13 13:45:00,2.0198,2.0355,2.0196,2.0295,1762917.7000
...,...,...,...,...,...
2021-08-26 09:35:00,2.5950,2.5970,2.5850,2.5870,610883.2000
2021-08-26 09:40:00,2.5860,2.5880,2.5820,2.5870,571969.7000
2021-08-26 09:45:00,2.5860,2.5870,2.5690,2.5780,1485829.2000


In [11]:
features = ['account', 'open', 'high', 'low', 'close',
           'r', 'm', 'v', 'min', 'max', 'mami', 'mac', 'mic', 'atr', 'atr%',
           'avg_price', 'med_price', 'typ_price', 'wcl_price', 'upperband',
           'middleband', 'lowerband', 'sma_7', 'sma_25', 'sma_99', 'ema_7', 'ema_25', 'ema_99',
           'macd', 'macd_signal', 'macd_hist', 'rsi', 'cci', 'dx', 'units']

In [12]:
learn_env = CryptoEnv(initial_account=1e6, data=data, lag=lag, ptc=2.7e-4, mode='train', features=features)

In [13]:
test_env = CryptoEnv(initial_account=1e6, data=data, lag=lag, ptc=2.7e-4, mode='test', features=features)

In [14]:
# learn_env.draw_cumulative_return(agent.model)

In [15]:
# test_env.draw_cumulative_return(agent.model)

In [16]:
# initialize ray
# https://ray.readthedocs.io/en/latest/package-ref.html#ray.init
# ray.init(ignore_reinit_error=True, log_to_driver=False, dashboard_host='0.0.0.0')

In [17]:
ppo_config = DEFAULT_CONFIG.copy()
if n_gpus:
    ppo_config['num_gpus'] = n_gpus
    ppo_config['tf_session_args']['device_count']['GPU'] = n_gpus

ppo_config['num_workers'] = 1
ppo_config['num_sgd_iter'] = 2
ppo_config['sgd_minibatch_size'] = 128
ppo_config['lr'] = 0.0003
ppo_config['gamma'] = 0.99
ppo_config['model']['fcnet_hiddens'] = [64, 64]
ppo_config['timesteps_per_iteration'] = len(learn_env.data)
ppo_config['train_batch_size'] = 8000
ppo_config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
ppo_config['framework']='tf'
ppo_config['env'] = "TradingEnv"

In [18]:
def create_env(config):
    return learn_env

In [19]:
from ray.tune.registry import register_env

register_env("TradingEnv", create_env)

In [20]:
# agent = PPOTrainer(config=ppo_config)

In [21]:
# result = agent.train()

In [22]:
# result

In [23]:
# tune hyperparamters with grid search
# https://github.com/ray-project/ray/blob/master/python/ray/tune/tune.py
ray.init(ignore_reinit_error=True)
env_name = 'TradingEnv'
ppo_config = {
    "env": env_name,
    "num_workers": 1,
#     'model': {
#         'fcnet_hiddens': tune.grid_search([[128, 128],[64, 64],[32, 32]])
#     },        
#     'train_batch_size': 1000,
#     "lr": tune.grid_search([0.0003, 0.0001]),
#     'gamma': tune.grid_search([0.99, 0.999]),
#     'framework': 'tf',
    'num_gpus': 1#n_gpus  
}
                      
analysis = tune.run(
    "PPO",
    name='PPO',
    verbose=1,
    stop={"training_iteration": 10000},
    config = ppo_config,
    checkpoint_freq=10,
    checkpoint_at_end=True,
    checkpoint_score_attr='episode_reward_mean',
    local_dir=root_dir+'\\ray-results',
#     progress_reporter=CLIReporter(),
    num_samples=1,  # for grid search, number of times to run each hyperparameter combo
    #     with_server=True,
    #     server_port=8267,
)


(pid=15352) Windows fatal exception: access violation
(pid=15352) 
2021-08-27 03:14:33,746	ERROR tune.py:546 -- Trials did not complete: [PPO_TradingEnv_e51fe_00000, PPO_TradingEnv_e51fe_00001]
2021-08-27 03:14:33,747	INFO tune.py:550 -- Total run time: 244.79 seconds (244.35 seconds for the tuning loop).
2021-08-27 03:14:33,747	WARNING tune.py:555 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
